# Libraries

In [ ]:
import re
import string
import csv
from joblib import dump, load

import numpy as np

import pandas as pd
#pd.set_option('display.max_colwidth', None)

import matplotlib.pyplot as plt
%matplotlib inline

import plotly.express as px

import nltk
from nltk import SnowballStemmer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize

from wordcloud import WordCloud

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer


# Loading data

In [ ]:
df = pd.read_csv('../data/SMSSpamCollection.txt', sep=',\s+',quoting=csv.QUOTE_ALL, delimiter='\t', names= ['target','message'])

In [ ]:
df['target'] = df['target'].apply(lambda row: row.strip())

#### Replace special characters (`&gt;` , `&lt;`, `<DECIMAL>`...)

In [ ]:
special_chars = {
    "&quot;" : '"',
    "&apos;" : "'",
    "&amp;" : "&",
    "&lt;" : "<",
    "&gt;" : ">"
}
df = df.replace({'message': special_chars}, regex=True)

In [ ]:
brackets = set()
def find_brackets(row):
    global brackets
    for bracket in re.findall(r'<[a-zA-Z0-9\s#]+>+', row):
        brackets.add(bracket)

df['message'].apply(find_brackets)
print(brackets)

In [ ]:
special_chars = {
    "<DECIMAL>" : '123',
    "<UKP>" : "£",
    "<EMAIL>" : "abc@email.com",
    "<fone no>" : "<phone 0123456789>",
    "<URL>" : "https://www.abcd.com",
    "<#>" : "321",
    "<TIME>" : "13:00"
}
df = df.replace({'message': special_chars}, regex=True)

In [ ]:
df.shape

- Our dataset contains 5574 observation and 2 characterisitics (which one is dependant variable, the other one is an independant variable)

In [ ]:
df.info()

- The dataset consists of 5574 English messages, each entry is designated as being ham or spam. Dataframe has two columns: 
 - The first column `target` indicating the class of message as ham or spam.
 - The second column `message` is the content of the message.

In [ ]:
df.sample(10, random_state=20)

In [ ]:
df.groupby('target').describe()

- Our dataset contains 4518 (out of 4827) unique `ham` messages and 653 (out of 747) unique `spam` ones.
- `Sorry, I'll call later` is the most common `ham` message.
- `Please call our customer service representative on FREEPHONE...` is the most common `spam` message.


# Feature engineering

We will be adding the following columns to the dataframe:
- `word_count` : The number of words
- `sentence_count` : The number of sentences
- `brackets_count` : The number of text between brackets
- `links_count` : The number of links
- `phone_count` : The number of phone numbers
- `money_count` : The number of money amounts

In [ ]:
class FeaturesExtractor(BaseEstimator, TransformerMixin):

    def __init__(self):
      nltk.download('punkt', quiet=True)
    
    def count_words(self, input_text):
        # remove punctuation, tokenize and return the number of tokens (words)
        message = input_text.translate(str.maketrans('', '', string.punctuation))
        return len(nltk.word_tokenize(input_text))

    def count_sentences(self, input_text):
        return len(sent_tokenize(input_text.lower()))

    def count_brackets(self, input_text):
        return len(re.findall(r'<[a-zA-Z0-9\s]+>+', input_text.lower()))

    def count_links(self, input_text):
        return len(re.findall(r'https?://\S+|www\.\S+', input_text.lower()))

    def count_phone(self, input_text):
        return len(re.findall(r'\d{5,}', input_text.lower()))

    def count_money(self, input_text):
        return len(re.findall(r'[$|£|€]\d+', input_text.lower()))+len(re.findall(r'\d+[$|£|€]', input_text.lower()))

    def transform(self, df, y=None):
        """The workhorse of this feature extractor"""
        df['word_count'] = df.message.apply(self.count_words)
        df['sentence_count'] = df.message.apply(self.count_sentences)
        df['brackets_count'] = df.message.apply(self.count_brackets)
        df['links_count'] = df.message.apply(self.count_links)
        df['phone_count'] = df.message.apply(self.count_phone)
        df['money_count'] = df.message.apply(self.count_money)
        return df
    def fit(self, df, y=None):
        """Returns `self` unless something different happens in train and test"""
        return self


In [ ]:
tc = FeaturesExtractor()

df = tc.fit_transform(df)

df.sample(10, random_state=20)

# EDA (Exploratory data analysis)

In [ ]:
df.describe()

- The average number of words is (`mean`) 18 words
- The average number of sentences is (`mean`) 2 sentences
- The mean values are close to medians
- There's a large difference  between the 75% and max value (there might be outliers in the dataset)

- The longest message (in terms of `word_count`) contains 196 (`max`) words.

In [ ]:
df.loc[df['word_count'] == df['word_count'].max()]

- The shortest message contains 1 word

In [ ]:
df.loc[df['word_count'] < 2].sort_values('word_count').head(5)

## Detecting outliers using the 1.5 IQR (Interquartile Rule)

### Applying the 1.5 IQ rule to `word_count`
- IQR = Q3-Q1 = 16 | Q1 = 7 | Q3 = 23
- IQR * 1.5 = 24
- IQR * 1.5 - Q1 = 17
- Q3 + IQR * 1.5 = 47

In [ ]:
wc_desc = df.describe()['word_count']
wc_Q1, wc_Q3 = wc_desc['25%'], wc_desc['75%']
IQR = wc_Q3 - wc_Q1
low, high = IQR * 1.5 - wc_Q1 , wc_Q3 + IQR * 1.5
print(f'Low outliers count : {len(df[df.word_count<low])}\nHigh outliers count : {len(df[df.word_count>high])}')

- Removing low outliers will result in removing more than **half** of the data, while high outliers count is low so it will not affect our data.

### Applying the 1.5 IQ rule to `sentence_count`
- IQR = Q3-Q1 = 16 | Q1 = 7 | Q3 = 23
- IQR * 1.5 = 24
- IQR * 1.5 - Q1 = 17
- Q3 + IQR * 1.5 = 47

In [ ]:
wc_desc = df.describe()['sentence_count']
wc_Q1, wc_Q3 = wc_desc['25%'], wc_desc['75%']
IQR = wc_Q3 - wc_Q1
low, high = IQR * 1.5 - wc_Q1 , wc_Q3 + IQR * 1.5
print(f'Low outliers count : {len(df[df.sentence_count<low])}\nHigh outliers count : {len(df[df.sentence_count>high])}')

- There are no lower outliers in the dataset, while there're 662 high outliers. Like `word_count`, high outliers count is low so they will not affect the data.

### Pairplot of the dataset

In [ ]:
fig = px.scatter_matrix(df,
                        dimensions=df.drop(['target','message'],axis=1).columns,
                        color="target",
                        height=700)
fig.show()

## Count of every class

In [ ]:
counts = df.groupby('target').count().reset_index()
counts = counts.rename(columns = {"message":"count"})
fig = px.bar(counts,
             x='target',
             y='count',
             color='target',
             width = 500, height=400)
fig.update_layout(title_text='Count of ham and spam messages in the dataset',
                  xaxis_title_text='Class',
                  yaxis_title_text='Count')
fig.show()

- The dataset contains **4827** `ham` messages, and **747** `spam` ones
- The dataset is imbalanced

## Word count distribution

In [ ]:
fig_hist = px.histogram(df,
                        x="word_count",
                        color="target",
                        barmode="overlay",
                        marginal="violin", #box or rug
                        hover_data=df.columns,
                        range_x=["0","100"],
                        width=700)
fig_hist.update_layout(title_text='Message length distribution (Word count)',
                       xaxis_title_text='Word Count',
                       bargap=0.3)
fig_hist.show()

- As we can see, `ham` messages lenght tends to be lower than `spam` messages lenght.

## Sentence count distribution

In [ ]:
fig_hist = px.histogram(df,
                        x="sentence_count",
                        color="target",
                        barmode="overlay",
                        marginal="violin", #box or rug
                        hover_data=df.columns,
                        range_x=["0","40"],
                        width=700)
fig_hist.update_layout(title_text='Message length distribution (Sentence count)',
                       xaxis_title_text='Sentence Count',
                       bargap=0.3)
fig_hist.show()

## Data pre-processing

In [ ]:
class ColumnExtractor(TransformerMixin, BaseEstimator):
    def __init__(self, cols):
        self.cols = cols

    def transform(self, X, **transform_params):        
        return X[self.cols]

    def fit(self, X, y=None, **fit_params):
        return self

### Text cleaning

Before we start using the messages we clean them. We'll do the this in the class CleanText:

    - Set all words to lowercase
	- Replace text between brackets with 'bracketstext' (present in spam messages)
	- Replace money amounts ($123 or 1£) with 'moneytext'
	- Replace remaining currency symbols with 'currsymb'
	- Replace links with 'weblink'
	- Replace phone numbers with 'phonenumber'
	- Replace punctuation with whitespace
	- Remove extra whitespaces
    - Remove remaining digits
    - Remove stopwords
    - Apply SnowballStemmer to keep the stem of words

In [ ]:
class CleanText(BaseEstimator, TransformerMixin):

	def __init__(self):
		nltk.download('stopwords', quiet=True)
		nltk.download('wordnet', quiet=True)
	
	def to_lower(self, input_text):
		return input_text.lower()

	def replace_brackets(self, input_text):
		# Replace text between brackets with 'bracketstext' (spam messages)
		return re.sub('<.*?>+', ' bracketstext ', input_text)

	def replace_money(self, input_text):
		# Replace money amounts ($123 or 1£) with 'moneytext'
		input_text = re.sub(r'[$|£|€]\d+', ' moneytext ', input_text)
		return re.sub(r'\d+[$|£|€]', ' moneytext ', input_text)

	def replace_currency(self, input_text):
		# Replace remaining currency symbols with 'currsymb'
		return re.sub(r'[$|£|€]', ' currsymb ', input_text)

	def replace_urls(self, input_text):
		# Replace links with 'weblink'
		link_regex = r'(http(s)?:\/\/.)?(www\.)?[-a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0-9@:%_\+.~#?&//=]*)'
		link_regex1 = r'https?://\S+|www\.\S+'
		link_regex2 = r'http.?://[^\s]+[\s]?'
		return re.sub(link_regex1, ' weblink ', input_text)

	def replace_phone_numbers(self, input_text):
		# Replace phone numbers with 'phonenumber'
		return re.sub(r'\d{5,}', ' phonenumber ', input_text)

	def replace_punctuation(self, input_text):
		# Replace punctuation with a space
		return input_text.translate(str.maketrans(dict.fromkeys(list(string.punctuation),' ')))
	
	def remove_extra_whitespace(self, input_text):
		# Remove extra whitespaces
		return re.sub(r'\s+', ' ', input_text)

	def remove_digits(self, input_text):
		return re.sub('\d+', '', input_text)

	def remove_stopwords(self, input_text):
		stop_words = stopwords.words('english')

		words = input_text.split()
		clean_words = [word for word in words if word not in stop_words]
		return ' '.join(clean_words)

	def apply_stemming(self, input_text):
		stemmer = SnowballStemmer('english')
		
		words = input_text.split()
		stemmed_words = [stemmer.stem(word) for word in words]
		return ' '.join(stemmed_words)


	def fit(self, X, y=None, **fit_params):
		return self

	def transform(self, X, **transform_params):
		clean_X = X.apply(self.to_lower)
		clean_X = clean_X.apply(self.replace_brackets)
		clean_X = clean_X.apply(self.replace_money)
		clean_X = clean_X.apply(self.replace_currency)
		clean_X = clean_X.apply(self.replace_urls)
		clean_X = clean_X.apply(self.replace_phone_numbers)
		clean_X = clean_X.apply(self.replace_punctuation)
		clean_X = clean_X.apply(self.remove_extra_whitespace)
		clean_X = clean_X.apply(self.remove_digits)
		clean_X = clean_X.apply(self.remove_stopwords)
		clean_X = clean_X.apply(self.apply_stemming)
		return clean_X

One side-effect of text cleaning is that some rows do not have any words left in their text. To deal with these missing values, we impute them with some placeholder text like `no_text`.

In [ ]:
ct = CleanText()

clean_msg = ct.fit_transform(df.message)
empty = clean_msg == ''
print(f'{clean_msg[empty].count()} records have no words left after text cleaning')
clean_msg.loc[empty] = 'no_text'

df['message_clean'] = clean_msg

### Target Encoding (spam = 1 / ham = 0)

In [ ]:
encoder = LabelEncoder()
targets = encoder.fit_transform(df.target)

df['target_encoded'] = targets

### Count of spam messages containing phone numbers, weblinks and money

In [ ]:
num_phone = len(df.query("phone_count!=0 and target_encoded==1"))
num_link = len(df.query("links_count!=0 and target_encoded==1"))
num_money = len(df.query("money_count!=0 and target_encoded==1"))
num_brackets = len(df.query("brackets_count!=0 and target_encoded==1"))

print(f'There are {num_phone} rows containing phone numbers, {num_link} ones containing links, {num_money} ones containing money, and {num_brackets} ones containing brackets.')
print(f'out of {len(df[df.target_encoded==1])} spam message')

## Tokens Visualization

### Top words in ham messages

In [ ]:
wc = WordCloud(background_color='white', max_words=150)
wc.generate(' '.join(df['message_clean'][df['target']=='ham']))
plt.figure(figsize=(12, 8))
plt.title('Top words in HAM messages',fontdict={'size': 22,  'verticalalignment': 'bottom'})
plt.imshow(wc)
plt.axis('off')
plt.show()

### Top words in spam messages

In [ ]:
wc = WordCloud(background_color='white', max_words=150)
wc.generate(' '.join(df['message_clean'][df['target']=='spam']))
plt.figure(figsize=(12, 8))
plt.title('Top words in SPAM messages',fontdict={'size': 22,  'verticalalignment': 'bottom'})
plt.imshow(wc)
plt.axis('off')
plt.show()

- As we can notice, phone numbers and money amounts are mostly used in spam messages

## Splitting data into train and test

In [ ]:
X, y = df.drop(['target_encoded','target'], axis=1), df.target_encoded

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=37)

print('Train shapes:', X_train.shape, y_train.shape,'\nTest shapes:', X_test.shape, y_test.shape)

## Vectorization

In [ ]:
textcountscols = ['word_count', 'sentence_count', 'brackets_count', 'links_count', 'phone_count', 'money_count']    

def grid_vect(clf, clf_params, X_train, y_train, X_test, y_test, parameters_text=None, vectorizer=None, cv_nb=5):
    vect_pipe = Pipeline([
        ('cleantext', ColumnExtractor(cols='message_clean')),
        ('vect', vectorizer)
    ])

    col_ext = ColumnExtractor(cols=textcountscols)
    features = FeatureUnion([('textcounts', col_ext), ('pipe', vect_pipe)], n_jobs=1)
    
    pipeline = Pipeline([('features', features), ('clf', clf)])

    # Join both parameters dictionaries together
    parameters = dict()
    if parameters_text:
        parameters.update(parameters_text)
    parameters.update(clf_params)
    
    grid_search = GridSearchCV(pipeline, parameters, scoring='f1', n_jobs=-1, verbose=2, cv=cv_nb)
    
    print("Performing Grid Search...")
    grid_search.fit(X_train, y_train)
    print("Grid Search Results :")
    print(f"Best CV score: {grid_search.best_score_}")
    print("Best parameters :")

    best_parameters = grid_search.best_estimator_.get_params()
    for param_name in sorted(parameters.keys()):
        print(f"\t{param_name}: {best_parameters[param_name]}")

    GS_best_estimator = grid_search.best_estimator_
    print(f"Train score with best_estimator_: {GS_best_estimator.score(X_train, y_train):.3f}")
    print(f"Test score with best_estimator_: {GS_best_estimator.score(X_test, y_test):.3f}")

    print("Classification Report Test Data")
    print(classification_report(y_test, GS_best_estimator.predict(X_test), target_names=['ham','spam']))
    
    return grid_search

In [ ]:
# Parameter grid settings for the vectorizers
parameters_vect = {
    'features__pipe__vect__max_df': (0.25, 0.5, 0.75),
    'features__pipe__vect__ngram_range': ((1, 1), (1, 2)),
    'features__pipe__vect__min_df': (1, 2, 3)
}

# Parameter grid settings for KNN
parameters_knn = {
    'clf__n_neighbors': (3, 5, 7, 8, 9, 10, 25, 30, 50, 70, 74)
}

# Parameter grid settings for MultinomialNB
parameters_mnb = {
    'clf__alpha': (0.00001, 0.0001, 0.001, 0.01, 0.1, 0.2, 0.25, 0.3, 0.4, 0.5, 0.75, 0.8, 0.9, 1.0)
}

## Classifiers

In [ ]:
knn = KNeighborsClassifier()
mnb = MultinomialNB()

### CountVectorizer

In [ ]:
c_vect = CountVectorizer()

In [ ]:
# KNN x CountVectorizer
knn_countvect = grid_vect(knn, parameters_knn, X_train, y_train, X_test, y_test, parameters_text=parameters_vect, vectorizer=c_vect)

In [ ]:
# MultinomialNB x CountVectorizer
mnb_countvect = grid_vect(mnb, parameters_mnb, X_train, y_train, X_test, y_test, parameters_text=parameters_vect, vectorizer=c_vect)

### TF-IDF

In [ ]:
tfidf_vect = TfidfVectorizer()

In [ ]:
# KNN x TF-IDF
knn_tfidf = grid_vect(knn, parameters_knn, X_train, y_train, X_test, y_test, parameters_text=parameters_vect, vectorizer=tfidf_vect)

In [ ]:
# MultinomialNB x TF-IDF
mnb_tfidf = grid_vect(mnb, parameters_mnb, X_train, y_train, X_test, y_test, parameters_text=parameters_vect, vectorizer=tfidf_vect)

## Best model

In [ ]:
textcountscols = ['word_count', 'sentence_count', 'brackets_count', 'links_count', 'phone_count', 'money_count']    

vectorizer = CountVectorizer(max_df=0.25, min_df=1, ngram_range=(1,2))
clf_mnb = MultinomialNB(alpha=0.3) 

vect_pipe = Pipeline([('cleantext', ColumnExtractor(cols='message_clean')),('vect', vectorizer)])
col_ext = ColumnExtractor(cols=textcountscols)

features = FeatureUnion([('textcounts', col_ext), ('pipe', vect_pipe)], n_jobs=-1)
pipeline = Pipeline([('features', features), ('clf', clf_mnb)])

best_model = pipeline.fit(X_train, y_train)

In [ ]:
model_path = '../model/model.joblib'
dump(best_model, model_path)

In [ ]:
recall = cross_val_score(best_model, X_test, y_test, cv=10, scoring='recall')
print(f'Recall : Mean = {np.mean(recall)}')
print(recall)

print('-'*10)

precision = cross_val_score(best_model, X_test, y_test, cv=10, scoring='precision')
print(f'Precision : Mean = {np.mean(precision)}')
print(precision)

print('-'*10)

f1 = cross_val_score(best_model, X_test, y_test, cv=10, scoring='f1')
print(f'f1 : Mean = {np.mean(f1)}')
print(f1)


In [ ]:
def predict(model_path, data):
    df = pd.DataFrame(data, columns=['message'])
    tc = FeaturesExtractor()
    ct = CleanText()

    df_counts = tc.transform(df)
    df_clean = ct.transform(df.message)
    df = df_counts
    df['message_clean'] = df_clean
    
    model = load(model_path)
    predictions = model.predict(df).tolist()
    predictions = [ ['ham','spam'][prediction] for prediction in predictions]
    print('Model predictions:')
    for i in range(len(data)):
        print(f'{data[i]} : {predictions[i]}')


In [ ]:
my_msg = ["Hello, can you please call me later ?",
          "Lets meet 2morroww after class",
          "Congratulations! You've won a $1OOO Walmart gift card. Go to http://bit.ly/123456to claim now.",
          "Take 20% off your order with code THANKYOU."]

predict(model_path, my_msg)